7-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

In [2]:
pip install -U pandas-profiling

     |████████████████████████████████| 258 kB 531 kB/s eta 0:00:01
     |████████████████████████████████| 599 kB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 622 kB/s eta 0:00:011
     |████████████████████████████████| 3.1 MB 21.1 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 773 kB/s  eta 0:00:01
     |████████████████████████████████| 291 kB 5.2 MB/s eta 0:00:01


  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27084 sha256=eb226e525e1a22aa8d81888bc1412c0f4c5b3e33111c505031b65a8714670a32
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291991 sha256=b08a1f3ed47402206ad3b05311de80103e935db11678b6aec0de8f7c5780c861
  Stored in directory: /root/.cache/pip/wheels/16/72/b0/e5be34699908d9ff25dcb3debbb717987b766af61bbddffdfe
Successfully built htmlmin imagehash
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_rows=150
%matplotlib inline
import os

In [4]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
# rating 컬럼의 이름을 play_count로 바꿉니다.
ratings.rename(columns={'rating':'play_count'}, inplace=True)
print(ratings.shape)
ratings.head()

(836478, 4)


,user_id,movie_id,play_count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
print(movies.shape)
movies.head()

(3883, 3)


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# ratings 와 movies를 합쳐줍니다.
df = pd.merge(ratings, movies, on = 'movie_id') # 'movie_id' 를 기준으로 merge합니다.
print(df.shape)
df.head()

(836478, 6)


,user_id,movie_id,play_count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


# Data Profiling
#profile = df.profile_report()
#profile

Columns 분석 (step. 2-1)
user_id: 각 유저의 고유한 id 값입니다. 총 6,039명입니다.
movie_id: 각 영화의 고유한 id 값입니다. 총 3,628편입니다.
play_count: 각 영화에 대한 유저의 별점입니다. 이번 프로젝트에는 재생 횟수로 해석하여 사용합니다. 3회 이하의 수는 제외 했으며,
4(41.7% / 348,971번), 3(31.2% / 261,196번), 5(27.1% 226,310번) 순으로 분포되어 있습니다.
timestamp 1970년 1월 1일(Universal Time (UTC) of January 1, 1970) 이후의 시간정보
title: 각 영화의 제목
genre: 각 영화의 장르

In [9]:
#가장 인기있는 영화 순위 (step. 2-2)
top_mv_df = df.pivot_table(index = 'title', values = 'play_count', aggfunc = [np.sum, np.mean])
top_mv_df = top_mv_df.sort_values(by = ('sum','play_count'), ascending=False) # 'play_count', 'sum' 기준으로 내림차순 정렬
top_mv_df.head(100)

,sum,mean
,play_count,play_count
title,,
American Beauty (1999),14449,4.499844
Star Wars: Episode IV - A New Hope (1977),13178,4.528522
Star Wars: Episode V - The Empire Strikes Back (1980),12648,4.384055
Saving Private Ryan (1998),11348,4.431082
Star Wars: Episode VI - Return of the Jedi (1983),11303,4.161635
Raiders of the Lost Ark (1981),11179,4.520421
"Silence of the Lambs, The (1991)",11096,4.441954
"Matrix, The (1999)",10903,4.479458


In [10]:
#Build Recommender System (step. 3)
#임의의 유저 데이터 만들기 (step. 3-1)

# 선호하는 영화 5편을 선택합니다.
my_favorite = [2028, 318, 356, 1704,1721]

all_movie_name = []
for i in my_favorite:
    movie_name = list(df[df['movie_id'] == i]['title'].unique())
    all_movie_name.append(movie_name)
all_movie_name

[['Saving Private Ryan (1998)'],
 ['Shawshank Redemption, The (1994)'],
 ['Forrest Gump (1994)'],
 ['Good Will Hunting (1997)'],
 ['Titanic (1997)']]

In [11]:
# 선호하는 영화 5편을 선택합니다.
my_favorite = ['Saving Private Ryan (1998)',
               'Shawshank Redemption, The (1994)',
               'Forrest Gump (1994)',
               'Good Will Hunting (1997)',
               'Titanic (1997)']

# 'kj'라는 user_id가 위 영화를 5번씩 봤다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['kj']*5,'title':my_favorite,'play_count':[5]*5})

my_movielist


,user_id,title,play_count
0,kj,Saving Private Ryan (1998),5
1,kj,"Shawshank Redemption, The (1994)",5
2,kj,Forrest Gump (1994),5
3,kj,Good Will Hunting (1997),5
4,kj,Titanic (1997),5


In [12]:
# my_movielist 와 df 를 합쳐줍니다.
if not df.isin({'user_id':['kj']})['user_id'].any():  # user_id에 kj 이라는 데이터가 없다면
    df = df.append(my_movielist)                 # 위에 임의로 만든 my_movielist 데이터를 추가해 줍니다. 

print(df.shape)
df[df['user_id'] == 'kj']

(836483, 6)


,user_id,movie_id,play_count,timestamp,title,genre
0,kj,NaN,5,NaN,Saving Private Ryan (1998),NaN
1,kj,NaN,5,NaN,"Shawshank Redemption, The (1994)",NaN
2,kj,NaN,5,NaN,Forrest Gump (1994),NaN
3,kj,NaN,5,NaN,Good Will Hunting (1997),NaN
4,kj,NaN,5,NaN,Titanic (1997),NaN


In [13]:
# CSR matrix (step. 3-2)

# ratings의 user_id,movie_id에 누락된 값이 있습니다. 몇몇 영화는 아예 play_count에 포함되지 않았기 때문입니다. CSR matrix를 만들 때,
# 'row index exceeds matrix dimensions' 오류가 발생함으로, user_id, movie_id를 초기화하여 다시 값을 부여하겠습니다.

user_idx_to_unique = {v: k for k, v in enumerate(df.user_id.unique())}
movie_idx_to_unique = {v: k for k, v in enumerate(df.title.unique())}

In [14]:
user_idx_to_unique

{1: 0,
 2: 1,
 12: 2,
 15: 3,
 17: 4,
 18: 5,
 19: 6,
 24: 7,
 28: 8,
 33: 9,
 39: 10,
 42: 11,
 44: 12,
 47: 13,
 48: 14,
 49: 15,
 53: 16,
 54: 17,
 58: 18,
 59: 19,
 62: 20,
 80: 21,
 81: 22,
 88: 23,
 89: 24,
 95: 25,
 96: 26,
 102: 27,
 107: 28,
 120: 29,
 123: 30,
 131: 31,
 136: 32,
 139: 33,
 146: 34,
 150: 35,
 151: 36,
 155: 37,
 166: 38,
 169: 39,
 175: 40,
 178: 41,
 180: 42,
 190: 43,
 191: 44,
 192: 45,
 195: 46,
 196: 47,
 202: 48,
 208: 49,
 216: 50,
 224: 51,
 225: 52,
 227: 53,
 230: 54,
 231: 55,
 235: 56,
 236: 57,
 242: 58,
 245: 59,
 259: 60,
 261: 61,
 264: 62,
 265: 63,
 266: 64,
 270: 65,
 272: 66,
 281: 67,
 285: 68,
 287: 69,
 291: 70,
 292: 71,
 293: 72,
 294: 73,
 295: 74,
 297: 75,
 299: 76,
 300: 77,
 301: 78,
 306: 79,
 307: 80,
 308: 81,
 309: 82,
 312: 83,
 314: 84,
 315: 85,
 317: 86,
 318: 87,
 319: 88,
 326: 89,
 327: 90,
 329: 91,
 331: 92,
 332: 93,
 333: 94,
 334: 95,
 335: 96,
 339: 97,
 340: 98,
 349: 99,
 352: 100,
 353: 101,
 355: 102,
 357: 

In [16]:
temp_user_data = df.user_id.map(user_idx_to_unique.get).dropna()
temp_movie_data = df.title.map(movie_idx_to_unique.get).dropna()

In [17]:
temp_user_data

0       0
1       1
2       2
3       3
4       4
     ... 
0    6039
1    6039
2    6039
3    6039
4    6039
Name: user_id, Length: 836483, dtype: int64

In [18]:
df.user_id = temp_user_data
df.title = temp_movie_data

In [19]:
df.user_id

0       0
1       1
2       2
3       3
4       4
     ... 
0    6039
1    6039
2    6039
3    6039
4    6039
Name: user_id, Length: 836483, dtype: int64

In [20]:
from scipy.sparse import csr_matrix
num_user = df['user_id'].nunique()
num_movie = df['title'].nunique()

csr_data = csr_matrix((df.play_count, (df.user_id, df.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [21]:
num_user

6040

In [22]:
num_movie

3628

In [23]:
#MF model (step. 3-3)

from implicit.als import AlternatingLeastSquares

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

In [25]:
# 훈련된 모델이 'Saving Private Ryan (1998)'을 예측한 나의 선호도를 파악해봅시다.
kj, saving_private_ryan =  user_idx_to_unique['kj'], movie_idx_to_unique['Saving Private Ryan (1998)']
kj_vector, saving_private_ryan_vector = als_model.user_factors[kj], als_model.item_factors[saving_private_ryan]

In [26]:
kj_vector

array([ 0.54280734, -0.2622057 , -0.58142823,  0.06421683,  0.47733614,
        0.17653853, -0.19893883,  0.04604925,  0.2304026 ,  0.19615759,
        0.89935094, -0.01028621, -0.10473733,  0.81254417,  0.9733365 ,
       -0.91581136,  0.59676075,  0.24672458,  0.5247193 , -0.17108133,
       -0.060828  ,  0.8288312 ,  0.6896941 , -0.6900663 ,  0.27220842,
       -0.33431932,  0.03073039, -0.09167901,  0.4534842 ,  0.6804024 ,
        0.18589903, -0.15619984,  0.56839913, -0.8543174 , -0.48147774,
       -0.07659622,  0.0353893 , -0.32551774, -0.76036847,  0.14524592,
       -0.23924993, -0.2561144 , -0.5068319 ,  0.12884413, -1.4060891 ,
       -0.39654538,  0.09098975,  0.82848996, -0.04061026, -1.0581591 ,
        0.3094207 , -0.57880557,  0.42357647,  0.03232566,  0.33333662,
        0.39240023, -0.029327  ,  0.15835993, -0.47822547, -0.04721342,
        0.13615729, -0.59636056,  0.29747528,  0.6741744 ,  0.1367727 ,
       -0.24345243, -0.31361237,  0.4654004 , -0.14048626,  0.03

In [27]:
saving_private_ryan_vector

array([-4.57129069e-03, -3.63203436e-02, -1.34436730e-02, -1.72453001e-02,
        7.90291888e-05,  6.10426394e-03, -2.13215448e-04, -6.70915470e-04,
        4.60074609e-03,  7.21925776e-03,  3.22861038e-02,  2.05098391e-02,
       -1.03366273e-02,  3.99378911e-02,  4.51814458e-02, -1.53874243e-02,
        1.71400793e-02, -1.15447473e-02,  1.87443346e-02,  1.89206246e-02,
        3.33133005e-02,  3.64488140e-02,  2.96161813e-03, -8.74520221e-04,
        2.80700903e-02,  2.80826148e-02,  2.84755994e-02,  3.26633733e-03,
        2.79688481e-02,  4.07290682e-02,  9.81368776e-03,  3.12410817e-02,
        2.89295055e-02, -5.60640870e-03,  9.15232394e-03,  2.54553314e-02,
        8.60607903e-03, -5.02097188e-04, -1.74248498e-02,  1.71658453e-02,
        2.98977997e-02,  1.34420665e-02, -2.50851810e-02, -6.42889924e-03,
       -5.20998240e-03, -2.37620389e-03, -3.41742393e-03,  2.55234446e-02,
        1.91088095e-02, -9.18348227e-03,  2.29154695e-02, -9.20802820e-03,
        3.53743322e-02,  

In [29]:
# kj와 'Saving Private Ryan (1998)'를 내적하는 코드
np.dot(kj_vector, saving_private_ryan_vector) # 0.80가 나왔습니다.

0.71851414

In [30]:
# 훈련된 모델이 'Toy Story (1995)'을 예측한 나의 선호도를 파악해봅시다.
kj, toy_story =  user_idx_to_unique['kj'], movie_idx_to_unique['Toy Story (1995)']
kj_vector, toy_story_vector = als_model.user_factors[kj], als_model.item_factors[toy_story]

In [31]:
# kj와 'Toy Story (1995)''를 내적하는 코드
np.dot(kj_vector, toy_story_vector) # 0.21이 나왔습니다.

0.1701807

In [32]:
#선호하는 영화와 비슷한 영화 추천하기 (step. 3-4)

# 'Saving Private Ryan (1998)' 와 유사한 영화를 추천해보겠습니다.
favorite_movie = movie_idx_to_unique['Saving Private Ryan (1998)']
similar_movie = als_model.similar_items(favorite_movie, N=15)
similar_movie

[(48, 1.0000001),
 (87, 0.6964153),
 (23, 0.55390877),
 (157, 0.49792713),
 (487, 0.48583353),
 (141, 0.4126544),
 (124, 0.39743277),
 (248, 0.3894602),
 (3499, 0.34132737),
 (121, 0.32840103),
 (92, 0.32813686),
 (323, 0.32448816),
 (75, 0.31844515),
 (988, 0.31575018),
 (99, 0.31397563)]

In [33]:
# movie_id와 title 매칭하기
idx_to_movie = {v:k for k,v in movie_idx_to_unique.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Saving Private Ryan (1998)',
 'Braveheart (1995)',
 "Schindler's List (1993)",
 'Shawshank Redemption, The (1994)',
 'Boat, The (Das Boot) (1981)',
 'Fugitive, The (1993)',
 'Matrix, The (1999)',
 'Good Will Hunting (1997)',
 'Simon Sez (1999)',
 'Silence of the Lambs, The (1991)',
 'Terminator 2: Judgment Day (1991)',
 'Dead Man Walking (1995)',
 'Hunt for Red October, The (1990)',
 'Glory (1989)',
 'American Beauty (1999)']

In [34]:
# 'kj' 유저가 가장 좋아할 만한 영화들 추천 하기(step. 3-5)

user = user_idx_to_unique['kj']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

# movie_id와 title 매칭하기
idx_to_movie = {v:k for k,v in movie_idx_to_unique.items()}
[idx_to_movie[i[0]] for i in movie_recommended]

["Schindler's List (1993)",
 'Braveheart (1995)',
 'Silence of the Lambs, The (1991)',
 'Jerry Maguire (1996)',
 'Back to the Future (1985)',
 'American Beauty (1999)',
 'Apollo 13 (1995)',
 'Sixth Sense, The (1999)',
 'Pulp Fiction (1994)',
 'Groundhog Day (1993)',
 'GoodFellas (1990)',
 'As Good As It Gets (1997)',
 'Fargo (1996)',
 'Green Mile, The (1999)',
 'Rain Man (1988)',
 'Truman Show, The (1998)',
 'Dead Man Walking (1995)',
 'Boat, The (Das Boot) (1981)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Sling Blade (1996)']

In [35]:
# 'Saving Private Ryan (1998)'이 추천에 기여한 정도 알아보기
explain = als_model.explain(user, csr_data, itemid=saving_private_ryan)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Saving Private Ryan (1998)', 0.4785338250772646),
 ('Shawshank Redemption, The (1994)', 0.09968009667177516),
 ('Good Will Hunting (1997)', 0.0695796154083188),
 ('Forrest Gump (1994)', 0.05229359774320461),
 ('Titanic (1997)', 0.006298156427396534)]